In [25]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import geemap
import ee
import folium

ee.Authenticate()
ee.Initialize(project='twoearthengineproject')


In [2]:
physio_koshi = gpd.read_file(r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp")
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

In [3]:
# The basin data is already in vector format
basinATLAS = ee.FeatureCollection('WWF/HydroATLAS/v1/Basins/level07')

kosi_region = ee_physio_koshi # Assuming this is a defined ee.Geometry

# Simply clip the vector collection directly to your region of interest
koshi_basins = basinATLAS.filterBounds(kosi_region)



In [9]:
# Large rectangle covering Koshi Basin region (approximate bounding box)
roi = ee.Geometry.Rectangle([85.0, 26.0, 88.5, 29.5])

# Print and visualize in Earth Engine
print("ROI coordinates:", roi.getInfo())

ROI coordinates: {'type': 'Polygon', 'coordinates': [[[85, 26], [88.5, 26], [88.5, 29.5], [85, 29.5], [85, 26]]]}


In [26]:

# ---- INPUT: Multiple pour point coordinates ----
coords = [
    [86.93058173849101, 26.533420103187844],
    [86.92509230931958, 26.53203826401275],
    [86.93504866918286, 26.526969919337223],
    [86.92629393895825, 26.533266919922816]
]

# Create FeatureCollection of points with IDs
points = ee.FeatureCollection([
    ee.Feature(ee.Geometry.Point(lon, lat), {'id': idx+1})
    for idx, (lon, lat) in enumerate(coords)
])

# Load HydroATLAS Level 12 basins
basins = ee.FeatureCollection('WWF/HydroATLAS/v1/Basins/level12')

# Function to find all upstream basins for a given HYBAS_ID
def find_upstream(all_basins, start_id):
    ids_to_check = ee.List([start_id])
    all_ids = ee.List([start_id])

    def loop_function(_, loop_data):
        loop_data = ee.Dictionary(loop_data)
        ids_to_check = ee.List(loop_data.get('ids_to_check'))
        all_ids = ee.List(loop_data.get('all_ids'))

        upstream = all_basins.filter(ee.Filter.inList('NEXT_DOWN', ids_to_check))
        new_ids = upstream.aggregate_array('HYBAS_ID').distinct().removeAll(all_ids)

        return ee.Dictionary({
            'ids_to_check': new_ids,
            'all_ids': all_ids.cat(new_ids)
        })

    result = ee.Dictionary(ee.List.sequence(0, 50).iterate(loop_function, {
        'ids_to_check': ids_to_check,
        'all_ids': all_ids
    }))

    return all_basins.filter(ee.Filter.inList('HYBAS_ID', result.get('all_ids')))

# Map over points to get upstream basins for each
def process_point(feature):
    start_basin = basins.filterBounds(feature.geometry()).first()
    start_id = start_basin.getNumber('HYBAS_ID')
    upstream = find_upstream(basins, start_id)
    return upstream

# Merge all upstream basins from all points
all_upstream_basins = ee.FeatureCollection(
    ee.List(points.aggregate_array('id')).iterate(
        lambda pid, acc: ee.FeatureCollection(acc).merge(
            process_point(points.filter(ee.Filter.eq('id', pid)).first())
        ),
        ee.FeatureCollection([])
    )
).distinct('HYBAS_ID')

# ---- Visualization in Folium ---- #
def add_ee_layer(self, ee_object, vis_params, name):
    if isinstance(ee_object, ee.Image):
        map_id_dict = ee_object.getMapId(vis_params)
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Google Earth Engine',
            name=name,
            overlay=True,
            control=True
        ).add_to(self)
    elif isinstance(ee_object, ee.FeatureCollection):
        vis_params_fc = vis_params.copy()
        vis_params_fc.update({'opacity': 1})
        map_id_dict = ee_object.style(**vis_params_fc).getMapId({})
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr='Google Earth Engine',
            name=name,
            overlay=True,
            control=True
        ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create base map
m = folium.Map(location=[26.5334, 86.9305], zoom_start=12)

# DEM background
dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
m.add_ee_layer(dem, {'min':0, 'max':4000, 'palette':['blue','green','brown','white']}, 'DEM')

# Upstream basins
m.add_ee_layer(all_upstream_basins, {'color':'blue', 'fillColor':'00000000', 'width':2}, 'Upstream Basins')

# Points
for lon, lat in coords:
    folium.Marker([lat, lon], icon=folium.Icon(color='red')).add_to(m)

folium.LayerControl().add_to(m)
m.save("multi_upstream_basins.html")
print("Map saved to multi_upstream_basins.html")

# ---- OPTIONAL: Export as Shapefile ---- #
task = ee.batch.Export.table.toDrive(
    collection=all_upstream_basins,
    description='multi_upstream_basins_shp',
    folder='EarthEngine_Exports',
    fileFormat='SHP'
)
task.start()
print("Shapefile export started — check Google Drive > EarthEngine_Exports when done.")


TypeError: Collection.style() got an unexpected keyword argument 'opacity'

In [22]:
# Pour point coordinates (lon, lat)
lon, lat = 86.93058173849101, 26.533420103187844
point = ee.Geometry.Point([lon, lat])

# Load HydroATLAS Level 12 basins
basins = ee.FeatureCollection('WWF/HydroATLAS/v1/Basins/level12')

# Find the starting basin
start_basin = basins.filterBounds(point).first()
start_id = start_basin.getNumber('HYBAS_ID')

# Iterative upstream search function
def find_upstream(all_basins, start_id):
    ids_to_check = ee.List([start_id])
    all_ids = ee.List([start_id])

    def loop_function(_, loop_data):
        loop_data = ee.Dictionary(loop_data)
        ids_to_check = ee.List(loop_data.get('ids_to_check'))
        all_ids = ee.List(loop_data.get('all_ids'))

        upstream = all_basins.filter(ee.Filter.inList('NEXT_DOWN', ids_to_check))
        new_ids = upstream.aggregate_array('HYBAS_ID').distinct().removeAll(all_ids)

        return ee.Dictionary({
            'ids_to_check': new_ids,
            'all_ids': all_ids.cat(new_ids)
        })

    result = ee.Dictionary(ee.List.sequence(0, 50).iterate(loop_function, {
        'ids_to_check': ids_to_check,
        'all_ids': all_ids
    }))

    return all_basins.filter(ee.Filter.inList('HYBAS_ID', result.get('all_ids')))

# Get all upstream basins
all_upstream_basins = find_upstream(basins, start_id)

# ---- Export as Shapefile to Google Drive ---- #
task = ee.batch.Export.table.toDrive(
    collection=all_upstream_basins,
    description='upstream_basins_shapefile',
    folder='EarthEngine_Exports',   # Google Drive folder name
    fileFormat='SHP'
)

task.start()
print("Export started. Check your Google Drive > EarthEngine_Exports folder when complete.")


Export started. Check your Google Drive > EarthEngine_Exports folder when complete.
